In [1]:
import numpy as np
import codecs
import re
%matplotlib inline

# Read documents from collection
The documents in the Reuters collection are in SGML format. So first, we need to be able to read SGML files.

In [2]:
class TT:
    """Tag Tuple index names"""
    TAG = 0
    META = 1
    SUBTAGS = 2
    CONTENT = 3

def ReadDocumentsFromFile(path, preview=0):
    global parts
    global partsIndex

    def ParseSGML(returnTag=None, allowDuplicates=True, root=True):
        """Parses *parts* until returnTag or the end of *parts* reached; returns parsed content as a map"""
        global parts
        global partsIndex

        # Force allowDuplicates "True" since "False" isn't fully implemented
        allowDuplicates = True

        subTags = dict()
        content = ""

        while partsIndex < numParts:
            part = parts[partsIndex]
            partsIndex += 1

            # Remove excess whitespace
            part = " ".join(part.split())
            if len(part) == 0:
                continue

            # Process tags
            if part[0] == "<":
                assert(part[-1] == ">")

                # Ignore comments
                if part[1] == "!":
                    continue

                # Return contents when *returnTag* is reached
                if part[1] == "/":
                    tag = part[2:-1]
                    assert(tag == returnTag)
                    return (subTags, content)

                # Add new tags to the stack
                tagAndMeta = part[1:-1].split()
                tag = tagAndMeta[0]

                # Store key-value pairs associated with the tag (formatted "KEY1=VALUE1 KEY2=VALUE2 ...")
                tagMeta = dict([x.split("=") for x in tagAndMeta[1::]])

                # Collect content and tags within the current tag
                (tagSubTags, tagContent) = ParseSGML(
                    returnTag=tag,
                    allowDuplicates=False,
                    root=False
                )

                tagTuple = (
                    tag,
                    tagMeta,
                    tagSubTags,
                    tagContent
                )

                if allowDuplicates:
                    if tag not in subTags:
                        subTags[tag] = list()
                    subTags[tag].append(tagTuple)
                else:
                    subTags[tag] = tagTuple

            # Process text
            else:
                content += (" " if len(content) > 0 else "") + part

        return (subTags, content)


    # Create one big string
    document = ""
    #with open(path, "r") as file:
    with codecs.open(path, 'r', encoding='utf-8', errors='ignore') as file:
        for i, line in enumerate(file):
            document += line.replace('\n', ' ')

            # Show the first several lines of the file
            if i < preview:
                print('%d: %s' % (i, line), end="")

    # Show the last line as well
    if preview:
        print("...\n%d: %s" % (i, line))
    
    parts = re.split("(<.*?>)", document)
    numParts = len(parts)
    partsIndex = 0
    allData = ParseSGML()
    
    return allData[0]["REUTERS"]

Let's look through the data directory for .sgm files and extract their contents. Each file is supposed to contain 1000 documents.

In [3]:
def GetDataFilePaths(directory):
    from os import listdir
    from os.path import isfile, join
    paths = [join(directory, filePath) for filePath in listdir(directory) if isfile(join(directory, filePath)) and filePath[-4:] == ".sgm"]
    return paths

In [4]:
sgmlDocuments = []
previewLines = 20

filePaths = GetDataFilePaths("reuters21578/")
for filePath in filePaths:
    sgmlDocuments += ReadDocumentsFromFile(filePath, previewLines)
    previewLines = 0

0: <!DOCTYPE lewis SYSTEM "lewis.dtd">
1: <REUTERS TOPICS="YES" LEWISSPLIT="TRAIN" CGISPLIT="TRAINING-SET" OLDID="8914" NEWID="4001">
2: <DATE>11-MAR-1987 18:04:17.59</DATE>
3: <TOPICS></TOPICS>
4: <PLACES><D>canada</D></PLACES>
5: <PEOPLE></PEOPLE>
6: <ORGS></ORGS>
7: <EXCHANGES></EXCHANGES>
8: <COMPANIES></COMPANIES>
9: <UNKNOWN> 
10: &#5;&#5;&#5;M
11: &#22;&#22;&#1;f0849&#31;reute
12: d f BC-INCO-SEES-NO-MAJOR-IM   03-11 0133</UNKNOWN>
13: <TEXT>&#2;
14: <TITLE>INCO SEES NO MAJOR IMPACT FROM DOW REMOVAL</TITLE>
15: <DATELINE>    TORONTO, March 11 - </DATELINE><BODY>Inco Ltd said it did not expect its
16: earlier reported removal from the Dow Jones industrial index to
17: make a major impact on the company's stock.
18:     "We don't think that individuals or institutions buy our
19: shares because we were one of the Dow Jones industrials,"
...
32587: </REUTERS>



In [5]:
numDocuments = len(sgmlDocuments)
print("Extracted %s documents from %d files" % ("{:,}".format(numDocuments), len(filePaths)))

Extracted 21,578 documents from 22 files


# Make an index
P115: Scoring algorithm
  1. "store N/df_t  at the head of the postings for t" to compute idf_t
  1. "store the term frequency tf_t,d for each postings entry"
  1. Use a heap as a priority queue for document scores

## Build an index

In [6]:
from collections import OrderedDict 

def ContentTokens(content):
    # Replace symbols and numbers with spaces
    content = "".join([(c if c.isalpha() else " ") for c in content if c != "'"])
    allTokens = content.split()
    return [x.lower() for x in allTokens if len(x) > 1 and x.replace("'", "a").isalpha()]

def CombinedTermCounts(a, b):
    for term, count in b.items():
        a[term] = (a[term] if term in a else 0) + count
    return a

def GetTermCounts(tagTuple):
    # Get term counts for this tag's content
    tokens = ContentTokens(tagTuple[TT.CONTENT])
    terms = set(tokens)
    termCounts = dict([(x, tokens.count(x)) for x in terms])

    # Accumulate term counts from sub tags
    for _, subTagTuples in tagTuple[TT.SUBTAGS].items():
        for subTagTuple in subTagTuples:
            subTagTermCounts = GetTermCounts(subTagTuple)
            termCounts = CombinedTermCounts(termCounts, subTagTermCounts)

    return termCounts

class TermPostings:
    """Postings list with metrics for a single term"""
    def __init__(self):
        self.postingsList = dict()
        self.cf = 0
        self.df = 0
        self.idf = 0
        self.log_idf = 0

In [7]:
# For each term, count occurrences in all documents
postings = dict()
documentTermCounts = [dict()] * numDocuments
for i, doc in enumerate(sgmlDocuments):
    termCounts = GetTermCounts(doc)
    documentTermCounts[i] = termCounts

    for term, count in termCounts.items():
        if term not in postings:
            postings[term] = TermPostings()
        postings[term].postingsList[i] = (count)

# Calculate frequency metrics
for _, termPostings in postings.items():
    termPostings.cf = np.sum([x for x in termPostings.postingsList.values()])
    termPostings.idf = numDocuments / len(termPostings.postingsList)
    termPostings.log_idf = np.log(termPostings.idf)

# Sort by term
index = OrderedDict([(x, postings[x]) for x in sorted(postings)])

## Build document vectors
To make vector scoring possible, we create a normalized vector representation for each document. Indexing and general performance improvements for document vector usage are not covered until chapter 7, so for now we're using brute force methods.

Document vectors can be created by normalizing the document term count vectors -- we carry over the (term, count) format.

In [8]:
class WeightsVector:
    """Vector representation of a document or query"""
    def __init__(self, termWeights):
        self.termWeights = termWeights
        self.length = 1

    def CosineSimilarity(self, vector):
        # It's more efficient if the shorter vector does the calculations
        if len(self.termWeights) > len(vector.termWeights):
            return vector.CosineSimilarity(self)

        # Dot product
        dot = np.sum([self.termWeights[x] * vector.termWeights[x] for x in self.termWeights if x in vector.termWeights])
        return dot / (self.length * vector.length)

    @staticmethod
    def FromString(string):
        # Extract tokens and count terms
        tokens = ContentTokens(string)
        terms = set(tokens)
        termCounts = dict([(x, tokens.count(x)) for x in terms])
        return WeightsVector.FromTermCounts(termCounts)        

    @staticmethod
    def FromTermCounts(termCounts):
        # Normalize term counts
        vectorLength = np.sqrt(np.sum([np.square(x) for x in termCounts.values()]))
        termWeights = dict([
            (termCount[0], termCount[1] / vectorLength)
            for termCount in termCounts.items()
        ])
        return WeightsVector(termWeights)


In [9]:
documentVectors = [None] * numDocuments
for i, counts in enumerate(documentTermCounts):
    documentVectors[i] = WeightsVector.FromTermCounts(counts)

## Run some queries using vector space scoring

In [10]:
def GetSimilarVectors(weightsVector):
    # Score each document vector
    scores = dict()
    for i, documentVector in enumerate(documentVectors):
        scores[i] = weightsVector.CosineSimilarity(documentVector)
    
    # Sort by highest score
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

def GetQueryVector(query):
    return WeightsVector.FromString(query)

def GetQueryScores(query):
    queryVector = GetQueryVector(query)
    return GetSimilarVectors(queryVector)

def GetBestResult(query):
    bestMatch = GetQueryScores(query)[0]
    if (bestMatch[1] > 0):
        return (bestMatch[0], bestMatch[1])
    else:
        print("No documents contain any of the queried terms " + str(ContentTokens(query)))
        return None

In [11]:
(docID, score) = GetBestResult("brazilian cocoa")
print("Score: %.2f" % score)
sgmlDocuments[docID]

Score: 0.58


('REUTERS',
 {'CGISPLIT': '"TRAINING-SET"',
  'LEWISSPLIT': '"TRAIN"',
  'NEWID': '"14468"',
  'OLDID': '"3451"',
  'TOPICS': '"BYPASS"'},
 {'COMPANIES': [('COMPANIES', {}, {}, '')],
  'DATE': [('DATE', {}, {}, '7-APR-1987 15:19:48.23')],
  'EXCHANGES': [('EXCHANGES', {}, {}, '')],
  'ORGS': [('ORGS', {}, {}, '')],
  'PEOPLE': [('PEOPLE', {}, {}, '')],
  'PLACES': [('PLACES', {}, {}, '')],
  'TEXT': [('TEXT',
    {'TYPE': '"UNPROC"'},
    {},
    "&#2; U.S. COCOA MERCHANT SPOT PRICES - APRIL 7 In U.S. dlrs per tonne for minimum truckload quantities ex-dock or ex-warehouse U.S. Eastern seaboard north of Hatteras seller's option: Main Crop Ivory Coast cocoa beans 2,148N Superior Bahia cocoa beans 2,062N Sanchez FAQ cocoa beans 1,945N Superior seasons Arriba cocoa beans 1,965N Ecuadorean cocoa liquor 2,541N Brazilian cocoa liquor 2,612N Pure pressed cocoa butter/African 4,646N PURE PRESSED COCOA BUTTER/OTHER 4,620N Cocoa press cake - natural 10/12 pct cocoa butter contents 975N All prices

In [12]:
(docID, score) = GetBestResult("Kidder Peabody")
print("Score: %.2f" % score)
sgmlDocuments[docID]

Score: 0.55


('REUTERS',
 {'CGISPLIT': '"TRAINING-SET"',
  'LEWISSPLIT': '"TRAIN"',
  'NEWID': '"2105"',
  'OLDID': '"18523"',
  'TOPICS': '"NO"'},
 {'COMPANIES': [('COMPANIES', {}, {}, '')],
  'DATE': [('DATE', {}, {}, '5-MAR-1987 10:48:58.69')],
  'EXCHANGES': [('EXCHANGES', {}, {}, '')],
  'ORGS': [('ORGS', {}, {}, '')],
  'PEOPLE': [('PEOPLE', {}, {}, '')],
  'PLACES': [('PLACES', {}, {}, '')],
  'TEXT': [('TEXT',
    {'TYPE': '"BRIEF"'},
    {'TITLE': [('TITLE',
       {},
       {},
       'KIDDER PEABODY ANALYST RAISES ESTIMATES, RECOMMENDATION ON PEPSICO')]},
    '&#2; ****** Blah blah blah. &#3;')],
  'TOPICS': [('TOPICS', {}, {}, '')],
  'UNKNOWN': [('UNKNOWN',
    {},
    {},
    '&#5;&#5;&#5;F &#22;&#22;&#1;f0408&#31;reute b f BC-******KIDDER-PEABODY 03-05 0012')]},
 '')

**Let's look at titles of the articles that are matching our queries.**

In [13]:
def GetDocumentMetadata(document, tag):
    for _, subTagTuples in document[TT.SUBTAGS].items():
        for subTagTuple in subTagTuples:
            if subTagTuple[TT.TAG] == tag and len(subTagTuple[TT.CONTENT]) > 0:
                return subTagTuple[TT.CONTENT]
            metadata = GetDocumentMetadata(subTagTuple, tag)
            if len(metadata) > 0:
                return metadata
    return ""

def PrintTopMatches(matches, count, meta='TITLE'):
    print("DocID\tScore\t%s\n" % meta.capitalize())
    for match in matches[:count]:
        docID = match[0]
        score = match[1]
        metadata = GetDocumentMetadata(sgmlDocuments[docID], meta) if len(meta) > 0 else ""
        print("%d\t%.2f\t%s" % (docID, score, metadata))

In [14]:
matches = GetQueryScores("brazilian cocoa")
PrintTopMatches(matches, 10)

DocID	Score	Title

13045	0.58	
3505	0.37	COCOA BUFFER STOCK RULES EFFECTIVE IMMEDIATELY
3470	0.36	Cocoa Council agrees new buffer stock rules - delegates
1721	0.36	NEW YORK COCOA FUTURES EXPECTED TO OPEN LOWER
3490	0.35	COCOA BUFFER STOCK RULES TO TAKE EFFECT IMMEDIATELY - DELEGATES
1367	0.32	
16767	0.32	GHANA COCOA PURCHASES 1,323 TONNES IN LATEST WEEK, CUMULATIVE 216,095 TONNES - OFFICIAL
10652	0.31	BRAZIL COCOA EXPORTERS UNLIKELY TO LIMIT SALES
3585	0.30	COCOA COUNCIL MEETING ENDS AFTER AGREEING RULES
1620	0.30	


In [15]:
matches = GetQueryScores("Kidder Peabody")
PrintTopMatches(matches, 10)

DocID	Score	Title

8104	0.55	KIDDER PEABODY ANALYST RAISES ESTIMATES, RECOMMENDATION ON PEPSICO
11103	0.33	NOMURA SAYS IT SEEKS LINK WITH KIDDER PEABODY
18750	0.29	PEABODY HOLDING COMPLETES ACQUISITION
7826	0.28	KIDDER UNIT SELLS CMOS, INCLUDING FLOATERS
10954	0.25	KIDDER, PEABODY DEFENDS EMPLOYEE DRUG TESTING
5019	0.25	THREE TRADERS INDICTED FOR INSIDER TRADING
10150	0.23	AMPLICON INITIAL OFFERING AT 12.25 DLRS A SHARE
8115	0.21	PEPSICO &lt;PEP> UPGRADED BY KIDDER PEABODY
18685	0.20	PROPOSED OFFERINGS RECENTLY FILED WITH THE SEC
16164	0.19	JOY TECHNOLOGIES SELLS 12-YEAR DEBENTURES


In [16]:
matches = GetQueryScores("car automobile gasoline gas industry ford")
PrintTopMatches(matches, 10)

DocID	Score	Title

2831	0.26	FORD &lt;F> MAY CAR OUTPUT UP 2.2 PCT
9529	0.24	FORD LATE FEBRUARY U.S. CAR SALES UP 29.3 PCT
2823	0.24	FORD MAY N. AMERICAN CAR PRODUCTION 213,790, UP 2.2 PCT FROM 209,109
9645	0.23	FORD CANADA FEBRUARY CAR SALES OFF SIX PCT
12378	0.23	FORD MID-MARCH CAR SALES UP 15.2 PCT
9626	0.23	FORD CANADA &lt;FC> FEBRUARY CAR SALES OFF SIX PCT
18048	0.22	TENNECO &lt;TGT> TO TRANSPORT GAS ON OPEN ACCESS
833	0.22	LEAR PETROLEUM &lt;LPT> CONSOLIDATES GAS UNITS
17660	0.21	JAPAN FIRMS TO LAUNCH SALES OF 100 OCTANE GASOLINE
18936	0.20	FORD &lt;F> MARCH U.S. CAR PRODUCTION UP


In [17]:
matches = GetQueryScores("car automobile gasoline gas industry toyota")
PrintTopMatches(matches, 10)

DocID	Score	Title

1060	0.23	TOYOTA ANNOUNCES HIGHER DOMESTIC CAR SALES
18048	0.22	TENNECO &lt;TGT> TO TRANSPORT GAS ON OPEN ACCESS
833	0.22	LEAR PETROLEUM &lt;LPT> CONSOLIDATES GAS UNITS
9670	0.21	TOYOTA MOTOR U.S.A. FEBRUARY CAR SALES DOWN
17660	0.21	JAPAN FIRMS TO LAUNCH SALES OF 100 OCTANE GASOLINE
6867	0.20	TOYOTA &lt;TOYOY> HIKES 1987 PRICES
4241	0.20	TOYOTA LIKELY TO NAME AMERICAN FIRMS AS SUPPLIERS
10321	0.19	U.S.SENATE LIFTS SOME BANS ON NATURAL GAS
2360	0.19	MORGAN STANLEY GROUP &lt;MS> UNIT IN GAS DEAL
16494	0.19	BROOKLYN UNION&lt;BU> SEEN HURT BY PIPELINE CLOSURE


In [18]:
matches = GetQueryScores("\"soybeans\" counts more for scoring than these other words do")
PrintTopMatches(matches, 10)

DocID	Score	Title

11097	0.21	DOW JONES INDUSTRIAL AVERAGE DOWN MORE THAN 13.2 PCT, EXCEEDS PERCENTAGE DROP IN 1929
15994	0.18	ASCS TAKES STEPS TO EASE SALES OF CCC SOYBEANS
9550	0.18	CEREALS MCAS TO BE UNCHANGED NEXT WEEK
13340	0.18	HOUSE COMMITTEE SLASHES "STAR WARS" FUNDING
21439	0.18	CEREALS MCAS TO BE UNCHANGED NEXT WEEK
548	0.18	U.S. MARKET LOAN NOT THAT ATTRACTIVE-BOSCHWITZ
3704	0.17	JAPAN BUYS LARGE AMOUNT OF BRAZILIAN SOYBEANS
20090	0.17	CEREALS MCAS TO BE UNCHANGED NEXT WEEK
18810	0.17	POLL FINDS MOST THINK HOSTILE TAKEOVERS HARMFUL
9404	0.17	CBT TRADERS LOOK AHEAD TO SPRING PLANTINGS


**We can also look for documents that are similar to each other.**

In [19]:
docID = 1252 # Some random document
document = sgmlDocuments[docID]
GetDocumentMetadata(document, 'TITLE')

'BRITISH MINISTER GOES TO TOKYO ASKING FREER ACCESS'

In [20]:
documentVector = documentVectors[docID]
matches = GetSimilarVectors(documentVector)
PrintTopMatches(matches, 10)

DocID	Score	Title

1252	1.00	BRITISH MINISTER GOES TO TOKYO ASKING FREER ACCESS
1033	0.82	UK MINISTER LOOKS TO EASE TENSION ON TOKYO TRIP
3454	0.81	UK MAY REVOKE JAPANESE FINANCIAL LICENSES
18602	0.81	TOSHIBA COULD BE FIRST TO FEEL U.K. TRADE ANGER
13521	0.79	BRITAIN, JAPAN CLASH OVER STOCK MARKET ACCESS
1648	0.77	BRITAIN WARNS JAPAN OVER TRADE ROW
19519	0.77	UK HOPES HOWARD'S TOKYO TRIP WILL EASE TENSIONS
20775	0.76	EUROPE ON SIDELINES IN U.S-JAPAN MICROCHIP ROW
5094	0.76	BRITISH MINISTER SAYS HE WARNED TOKYO OF SANCTIONS
3622	0.76	NAKASONE SOUNDS CONCILIATORY NOTE IN CHIP DISPUTE
